## RUN

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from itertools import chain, repeat

In [2]:
df = pd.read_csv("results/holdout_all-tp_run.csv")
DF = df[df['tp'] == 'fu3']

In [3]:
DF2 = DF[['model', 'test_ids', 'test_lbls', 'test_probs',
         'holdout_ids', 'holdout_lbls', 'holdout_preds']]

In [4]:
test_id = sum([DF['test_ids'].values[i].strip("[]").split(', ') for i in range(28)], [])
test_probs = np.array([prob_lists for probs in DF["test_probs"] for prob_lists in eval(probs)])[:,1]
test_lbls = sum([DF['test_lbls'].values[i].strip("[]").split(', ') for i in range(28)], [])
test_model = list(chain.from_iterable((repeat(number, 650) for number in ['GB','LR','SVM-lin','SVM-rbf'])))

In [5]:
RUN = pd.DataFrame({
    "Model" : test_model,
    "Set" : 'Test set',
    "ID" : test_id,
    "Pred" : test_probs, 
    "True" : test_lbls,})
RUN_DF = RUN.astype({"ID":'int', "Pred":'float', "True":'float'})

In [6]:
RUN_DF

,Model,Set,ID,Pred,True
0,GB,Test set,215284,0.8881,0.0
1,GB,Test set,308867,0.9613,1.0
2,GB,Test set,469693,0.1427,1.0
3,GB,Test set,1781386,0.1823,1.0
4,GB,Test set,4908925,0.9799,0.0
...,...,...,...,...,...
2595,SVM-rbf,Test set,94345446,0.9292,1.0
2596,SVM-rbf,Test set,95263539,0.0405,0.0
2597,SVM-rbf,Test set,98537667,0.4776,1.0
2598,SVM-rbf,Test set,98600208,0.1032,0.0


In [ ]:
RUN_DF.to_csv('newlbls-fu3-espad-fu3-19a-binge-n650_val_RUN.csv', index=False)

In [7]:
# SVM-rbf
holdout_id = DF['holdout_ids'].values[21].strip("[]").split(', ')
holdout_probs = [b.split(', ')[1] for b in DF['holdout_preds'].values[21].strip("]]").split('], [')]
holdout_lbls = DF['holdout_lbls'].values[21].strip("[]").split(', ')
holdout_model = list(chain.from_iterable((repeat(number, 102) for number in ['SVM-rbf'])))

In [8]:
HOL = pd.DataFrame({
    "Model" : holdout_model,
    "Set" : 'Holdout set',
    "ID" : holdout_id,
    "Pred" : holdout_probs, 
    "True" : holdout_lbls,})
HOL_DF = HOL.astype({"ID":'int', "Pred":'float', "True":'float'})

In [14]:
HOL_DF

,Model,Set,ID,Pred,True
0,SVM-rbf,Holdout set,1163495,0.9696,1.0
1,SVM-rbf,Holdout set,1938036,0.9453,1.0
2,SVM-rbf,Holdout set,2103894,0.0086,1.0
3,SVM-rbf,Holdout set,2766073,0.9699,1.0
4,SVM-rbf,Holdout set,3504454,0.0931,0.0
...,...,...,...,...,...
97,SVM-rbf,Holdout set,95957207,0.3424,0.0
98,SVM-rbf,Holdout set,97739048,0.4355,0.0
99,SVM-rbf,Holdout set,99217838,0.4457,1.0
100,SVM-rbf,Holdout set,99677574,0.3094,0.0


In [9]:
HOL_DF.to_csv('newlbls-fu3-espad-fu3-19a-binge-n650_hol_RUN.csv', index=False)

In [10]:
RUN = pd.concat([RUN_DF, HOL_DF])

In [11]:
SVM = RUN.groupby('Model').get_group('SVM-rbf')

In [12]:
save_absolute_path = "/ritter/share/data/IMAGEN/Instrument/CTQ.csv"
CTQ_DF = pd.read_csv(save_absolute_path, low_memory=False)

In [13]:
CTQ_DF

,ID,Session,Emotional abuse sum,Physical abuse sum,Sexsual abuse sum,Emotional neglect sum,Physical neglect sum,Denial sum
0,112288,BL,0,0,0,16,8,11
1,215284,BL,5,0,0,18,8,10
2,240546,BL,0,0,0,20,8,12
3,297685,BL,0,0,0,20,8,12
4,308867,BL,0,0,0,20,8,12
...,...,...,...,...,...,...,...,...
5475,99747799,FU3,0,0,0,20,8,11
5476,99873252,FU3,4,0,0,14,4,5
5477,99875982,FU3,0,0,0,20,8,11
5478,99930021,FU3,0,0,0,20,8,11


In [ ]:
SVM

In [ ]:
DF = pd.merge(SVM, CTQ_DF, on='ID', how='inner')

In [ ]:
DF

In [ ]:
train = DF.groupby('Set').get_group('Test set')
test = DF.groupby('Set').get_group('Holdout set')
col = DF.columns[6:]

#### Training

In [ ]:
df = pd.read_csv(sorted(glob("results/newlbls-fu3-espad-fu3-19a-binge-*/*/run.csv"))[-1])

# This part need to be updated using directly h5files
CTQ = IMAGEN_instrument()
binge_CTQ = CTQ.read_instrument(
    'newlbls-fu3-espad-fu3-19a-binge-n650_CTQ.csv'
)

In [ ]:
test_id = sum([df['test_ids'][i].strip('][').split(', ') for i in range(28)], [])
test_probs = np.array([prob_lists for probs in df["test_probs"] for prob_lists in eval(probs)])[:,1]
test_lbls = sum([df['test_lbls'][i].strip('][').split(', ') for i in range(28)], [])
model = [g for (g, dfi) in df.groupby(["i", "o", "technique", "model"])]
test_model = list(chain.from_iterable((repeat(number, 650) for number in model)))
col_binge_CTQ = binge_CTQ.columns[:-6]

In [ ]:
df2 = pd.DataFrame({
    "ID" : test_id,
    "Model" : test_model,
    "Probability" : test_probs, 
    "Labels" : test_lbls,})
DF = df2.astype({"ID":'int', "Probability":'float', "Labels":'float'})

def models(x):
    if str(x) == "('X', 'Binge', 'cb', 'GB')":
        return 'GB'
    elif str(x) == "('X', 'Binge', 'cb', 'LR')":
        return 'LR'
    elif str(x) == "('X', 'Binge', 'cb', 'SVM-lin')":
        return 'SVM-lin'
    else:
        return 'SVM-rbf'

DF['Model'] = DF['Model'].apply(models)

In [ ]:
TP = (DF.Labels == 1.0) & (DF.Probability >= 0.5)
TN = (DF.Labels == 0.0) & (DF.Probability < 0.5)
FP = (DF.Labels == 0.0) & (DF.Probability >= 0.5)
FN = (DF.Labels == 1.0) & (DF.Probability < 0.5)

DF['TP prob'] = DF[TP]['Probability']
DF['TN prob'] = DF[TN]['Probability']
DF['FP prob'] = DF[FP]['Probability']
DF['FN prob'] = DF[FN]['Probability']
DF['T prob'] = DF[TP | TN]['Probability']
DF['F prob'] = DF[FP | FN]['Probability']

conditionlist = [
    (DF['TP prob'] >= 0.5) ,
    (DF['TN prob'] < 0.5) ,
    (DF['FP prob'] >= 0.5),
    (DF['FN prob'] < 0.5)]
choicelist = ['TP', 'TN', 'FP', 'FN']
DF['Prob'] = np.select(conditionlist, choicelist, default='Not Specified')

conditionlist2 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'TN'),
    (DF['Prob'] == 'FP') | (DF['Prob'] == 'FN')]
    
choicelist2 = ['TP & TN', 'FP & FN']
DF['Predict TF'] = np.select(conditionlist2, choicelist2, default='Not Specified')

conditionlist3 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FP'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FN')]

choicelist3 = ['TP & FP', 'TN & FN']
DF['Model PN'] = np.select(conditionlist3, choicelist3, default='Not Specified')


conditionlist4 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FN'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FP')]

choicelist4 = ['TP & FN', 'TN & FP']
DF['Label PN'] = np.select(conditionlist4, choicelist4, default='Not Specified')

In [ ]:
DF = pd.merge(DF, binge_CTQ[['ID','Site','Sex','Class']], on='ID', how = 'outer')

In [ ]:
DF.to_csv('newlbls-fu3-espad-fu3-19a-binge-n650_val_run.csv', index=False)

#### Holdout

In [ ]:
df = pd.read_csv("results/holdout_all-tp_run.csv")

# This part need to be updated using directly h5files
CTQ = IMAGEN_instrument()
binge_CTQ = CTQ.read_instrument(
    'newlbls-fu3-espad-fu3-19a-binge-n650_CTQ.csv'
)

In [ ]:
test_id = sum([df['test_ids'][i].strip('][').split(', ') for i in range(28)], [])
probs = sum([df['test_probs'][i].strip('][').split(', ') for i in range(28)], [])
test_probs = [p[:-1] for i, p in enumerate(probs) if i%2 == 1]
test_lbls = sum([df['test_lbls'][i].strip('][').split(', ') for i in range(28)], [])
model = [g for (g, dfi) in df.groupby(["io", "technique", "model"])]
test_model = list(chain.from_iterable((repeat(number, 620) for number in model)))
col_binge_CTQ = binge_CTQ.columns[:-6]

In [ ]:
df2 = pd.DataFrame({
    "ID" : test_id,
    "Model" : test_model,
    "Probability" : test_probs, 
    "Labels" : test_lbls,})
DF = df2.astype({"ID":'int', "Probability":'float', "Labels":'float'})

def models2(x):
    if str(x) == "('X-Binge', 'cb', 'GB')":
        return 'GB'
    elif str(x) == "('X-Binge', 'cb', 'LR')":
        return 'LR'
    elif str(x) == "('X-Binge', 'cb', 'SVM-lin')":
        return 'SVM-lin'
    else:
        return 'SVM-rbf'

DF['Model'] = DF['Model'].apply(models2)

In [ ]:
TP = (DF.Labels == 1.0) & (DF.Probability >= 0.5)
TN = (DF.Labels == 0.0) & (DF.Probability < 0.5)
FP = (DF.Labels == 0.0) & (DF.Probability >= 0.5)
FN = (DF.Labels == 1.0) & (DF.Probability < 0.5)

DF['TP prob'] = DF[TP]['Probability']
DF['TN prob'] = DF[TN]['Probability']
DF['FP prob'] = DF[FP]['Probability']
DF['FN prob'] = DF[FN]['Probability']
DF['T prob'] = DF[TP | TN]['Probability']
DF['F prob'] = DF[FP | FN]['Probability']

conditionlist = [
    (DF['TP prob'] >= 0.5) ,
    (DF['TN prob'] < 0.5) ,
    (DF['FP prob'] >= 0.5),
    (DF['FN prob'] < 0.5)]
choicelist = ['TP', 'TN', 'FP', 'FN']
DF['Prob'] = np.select(conditionlist, choicelist, default='Not Specified')

conditionlist2 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'TN'),
    (DF['Prob'] == 'FP') | (DF['Prob'] == 'FN')]
    
choicelist2 = ['TP & TN', 'FP & FN']
DF['Predict TF'] = np.select(conditionlist2, choicelist2, default='Not Specified')

conditionlist3 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FP'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FN')]

choicelist3 = ['TP & FP', 'TN & FN']
DF['Model PN'] = np.select(conditionlist3, choicelist3, default='Not Specified')


conditionlist4 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FN'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FP')]

choicelist4 = ['TP & TN', 'FP & FN']
DF['Label PN'] = np.select(conditionlist4, choicelist4, default='Not Specified')

In [ ]:
DF = pd.merge(DF, binge_CTQ[['ID','Site','Sex','Class']], on='ID', how = 'outer')

In [ ]:
DF.to_csv('newlbls-fu3-espad-fu3-19a-binge-n650_test_run.csv', index=False)

## 2. Generate the RUN file for posthoc-analysis
### 2.1. Generate the dataset RUN file
#### 2.1.1 Validation dataset result to RUN file
<b> Load the dataset <b/>

In [ ]:
df = pd.read_csv(sorted(glob("results/newlbls-fu3-espad-fu3-19a-binge-*/*/run.csv"))[-1])

# This part need to be updated using directly h5files
CTQ = IMAGEN_instrument()
binge_CTQ = CTQ.read_instrument(
    'newlbls-fu3-espad-fu3-19a-binge-n650_CTQ.csv'
)

<b> Generate the RUN dataframe </b>

In [ ]:
test_id = sum([df['test_ids'][i].strip('][').split(', ') for i in range(28)], [])
test_probs = np.array([prob_lists for probs in df["test_probs"] for prob_lists in eval(probs)])[:,1]
test_lbls = sum([df['test_lbls'][i].strip('][').split(', ') for i in range(28)], [])
model = [g for (g, dfi) in df.groupby(["i", "o", "technique", "model"])]
test_model = list(chain.from_iterable((repeat(number, 650) for number in model)))
col_binge_CTQ = binge_CTQ.columns[:-6]

In [ ]:
df2 = pd.DataFrame({
    "ID" : test_id,
    "Model" : test_model,
    "Probability" : test_probs, 
    "Labels" : test_lbls,})
DF = df2.astype({"ID":'int', "Probability":'float', "Labels":'float'})

def models(x):
    if str(x) == "('X', 'Binge', 'cb', 'GB')":
        return 'GB'
    elif str(x) == "('X', 'Binge', 'cb', 'LR')":
        return 'LR'
    elif str(x) == "('X', 'Binge', 'cb', 'SVM-lin')":
        return 'SVM-lin'
    else:
        return 'SVM-rbf'

DF['Model'] = DF['Model'].apply(models)

In [ ]:
TP = (DF.Labels == 1.0) & (DF.Probability >= 0.5)
TN = (DF.Labels == 0.0) & (DF.Probability < 0.5)
FP = (DF.Labels == 0.0) & (DF.Probability >= 0.5)
FN = (DF.Labels == 1.0) & (DF.Probability < 0.5)

DF['TP prob'] = DF[TP]['Probability']
DF['TN prob'] = DF[TN]['Probability']
DF['FP prob'] = DF[FP]['Probability']
DF['FN prob'] = DF[FN]['Probability']
DF['T prob'] = DF[TP | TN]['Probability']
DF['F prob'] = DF[FP | FN]['Probability']

conditionlist = [
    (DF['TP prob'] >= 0.5) ,
    (DF['TN prob'] < 0.5) ,
    (DF['FP prob'] >= 0.5),
    (DF['FN prob'] < 0.5)]
choicelist = ['TP', 'TN', 'FP', 'FN']
DF['Prob'] = np.select(conditionlist, choicelist, default='Not Specified')

conditionlist2 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'TN'),
    (DF['Prob'] == 'FP') | (DF['Prob'] == 'FN')]
    
choicelist2 = ['TP & TN', 'FP & FN']
DF['Predict TF'] = np.select(conditionlist2, choicelist2, default='Not Specified')

conditionlist3 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FP'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FN')]

choicelist3 = ['TP & FP', 'TN & FN']
DF['Model PN'] = np.select(conditionlist3, choicelist3, default='Not Specified')


conditionlist4 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FN'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FP')]

choicelist4 = ['TP & FN', 'TN & FP']
DF['Label PN'] = np.select(conditionlist4, choicelist4, default='Not Specified')

In [ ]:
DF = pd.merge(DF, binge_CTQ[['ID','Site','Sex','Class']], on='ID', how = 'outer')

<b> Save the RUN file

In [ ]:
DF.to_csv('newlbls-fu3-espad-fu3-19a-binge-n650_val_run.csv', index=False)

#### 2.2.1 Test dataset result to RUN file
<b> Load the dataset <b/>

In [ ]:
df = pd.read_csv("results/holdout_all-tp_run.csv")

# This part need to be updated using directly h5files
CTQ = IMAGEN_instrument()
binge_CTQ = CTQ.read_instrument(
    'newlbls-fu3-espad-fu3-19a-binge-n650_CTQ.csv'
)

<b> Generate the RUN dataframe </b>

In [ ]:
test_id = sum([df['test_ids'][i].strip('][').split(', ') for i in range(28)], [])
probs = sum([df['test_probs'][i].strip('][').split(', ') for i in range(28)], [])
test_probs = [p[:-1] for i, p in enumerate(probs) if i%2 == 1]
test_lbls = sum([df['test_lbls'][i].strip('][').split(', ') for i in range(28)], [])
model = [g for (g, dfi) in df.groupby(["io", "technique", "model"])]
test_model = list(chain.from_iterable((repeat(number, 620) for number in model)))
col_binge_CTQ = binge_CTQ.columns[:-6]

In [ ]:
df2 = pd.DataFrame({
    "ID" : test_id,
    "Model" : test_model,
    "Probability" : test_probs, 
    "Labels" : test_lbls,})
DF = df2.astype({"ID":'int', "Probability":'float', "Labels":'float'})

def models2(x):
    if str(x) == "('X-Binge', 'cb', 'GB')":
        return 'GB'
    elif str(x) == "('X-Binge', 'cb', 'LR')":
        return 'LR'
    elif str(x) == "('X-Binge', 'cb', 'SVM-lin')":
        return 'SVM-lin'
    else:
        return 'SVM-rbf'

DF['Model'] = DF['Model'].apply(models2)

In [ ]:
TP = (DF.Labels == 1.0) & (DF.Probability >= 0.5)
TN = (DF.Labels == 0.0) & (DF.Probability < 0.5)
FP = (DF.Labels == 0.0) & (DF.Probability >= 0.5)
FN = (DF.Labels == 1.0) & (DF.Probability < 0.5)

DF['TP prob'] = DF[TP]['Probability']
DF['TN prob'] = DF[TN]['Probability']
DF['FP prob'] = DF[FP]['Probability']
DF['FN prob'] = DF[FN]['Probability']
DF['T prob'] = DF[TP | TN]['Probability']
DF['F prob'] = DF[FP | FN]['Probability']

conditionlist = [
    (DF['TP prob'] >= 0.5) ,
    (DF['TN prob'] < 0.5) ,
    (DF['FP prob'] >= 0.5),
    (DF['FN prob'] < 0.5)]
choicelist = ['TP', 'TN', 'FP', 'FN']
DF['Prob'] = np.select(conditionlist, choicelist, default='Not Specified')

conditionlist2 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'TN'),
    (DF['Prob'] == 'FP') | (DF['Prob'] == 'FN')]
    
choicelist2 = ['TP & TN', 'FP & FN']
DF['Predict TF'] = np.select(conditionlist2, choicelist2, default='Not Specified')

conditionlist3 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FP'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FN')]

choicelist3 = ['TP & FP', 'TN & FN']
DF['Model PN'] = np.select(conditionlist3, choicelist3, default='Not Specified')


conditionlist4 = [
    (DF['Prob'] == 'TP') | (DF['Prob'] == 'FN'),
    (DF['Prob'] == 'TN') | (DF['Prob'] == 'FP')]

choicelist4 = ['TP & TN', 'FP & FN']
DF['Label PN'] = np.select(conditionlist4, choicelist4, default='Not Specified')

In [ ]:
DF = pd.merge(DF, binge_CTQ[['ID','Site','Sex','Class']], on='ID', how = 'outer')

<b> Save the RUN file

In [ ]:
DF.to_csv('newlbls-fu3-espad-fu3-19a-binge-n650_test_run.csv', index=False)